# Recuperação da Informação e Busca na Web - 2018.1

### Atividade: Lab 02 - Parte 1 - Expansão de Consultas
### Aluno: Johanny de Lucena Santos

## - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

## Importações necessárias para a análise dos dados

In [1]:
import pandas as pd
import nltk
import numpy as np
import ast
import re
from unicodedata import normalize
from collections import Counter
import math

from scipy import sparse
from nltk import bigrams
import scipy.sparse as sps

### Fazendo download de biblioteca necessária para realizar a tokenização das palavras

In [72]:
#nltk.download('punkt')



## Definindo o arquivo com os dados a ser analisado

In [2]:
dataset = pd.read_csv('../data/estadao_noticias_eleicao.csv', sep=',', encoding="utf-8")

#Substituindo linhas com valores NaN por string vazia, para não atrapalhar nas operações
dataset = dataset.replace(np.nan, '', regex=True)

### Função para remover acentuação das palavras dos documentos
Módulo **re** e **unicodedata** necessários para tratamento de expressões regulares e normalização de string

In [6]:
def remove_acentuacao(palavra):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    palavra = normalize('NFKD', palavra).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', palavra)

## Realizando junção dos títulos, subtítulos e conteúdos
Na junção, está sendo removidos todos os acentos para facilitar a criação dos tokens dos dados

In [7]:
documentos = dataset.titulo + " " + dataset.subTitulo + " " + dataset.conteudo
documentos = documentos.apply(lambda palavra: remove_acentuacao(palavra).lower())

## Matrix and Vocabulary construction

In [8]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

## Criação dos docIDs

In [9]:
docIDs = dataset.idNoticia

## Criando os tokens dos dados e a frequência de termos de cada documento da lista de postings
Utilizando a função **Count** da biblioteca **Collections** para realizar a contagem dos termos (frequência)

In [26]:
tokens = documentos.apply(nltk.word_tokenize)
tokens_list = documentos.apply(lambda text: text.lower().split())
tokens2 = [token for tokens_list in tokens_list for token in tokens_list]
termo_frequency = tokens.apply(Counter)

In [28]:
matrix, vocab = co_occurrence_matrix(tokens2)

## Consult bigram frequency

In [30]:
consultable_matrix = matrix.tocsr()

In [31]:
def consult_frequency(w1, w2):
    return (consultable_matrix[vocab[w1], vocab[w2]])

In [45]:
def top_3_words(termo):
    ranking = []
    for termo_matrix in consultable_matrix:
        ranking.append(consult_frequency(termo, termo_matrix))
    
    return sorted(ranking, reverse=True)[0:3]

## Função geradora do índice invertido

In [12]:
indiceInvertido = {}

def gerarIndiceInvertido():
    for i in range(len(tokens)):
        idNoticia = docIDs[i]
        palavras = tokens[i]
    
        for palavra in palavras:
            if palavra not in indiceInvertido:
                indiceInvertido[palavra.lower()] = {}
        
            if not indiceInvertido[palavra.lower()].get(idNoticia):
                docs = indiceInvertido[palavra.lower()]
                docs[idNoticia] = termo_frequency[i][palavra.lower()]

In [13]:
gerarIndiceInvertido()

## Função para retornar as palavras mais proximas

In [14]:
def busca_aproximada(termo, qtd):
    termos = indiceInvertido.keys()
    return sorted(termos, key=lambda palavra: nltk.edit_distance(termo.lower(),palavra))[0:qtd]

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

## Exemplo consult frequency

In [32]:
w1 = 'poucos'
w2 = 'recursos'
consult_frequency(w1,w2)

3